***
# メモ
* homeチームを選択できるようにする
* awayチームの"_グループ"を消す
* 目的 → いつどこの席が空席か いつどこの席がいくらの値段か この二つの情報が知りたい
* 3/12からの対象チーム → 琉球, 川崎, A千葉, 仙台
* 変化率を表す際に指数を用いるとか → https://bellcurve.jp/statistics/course/18948.html

*** 
# テスト

***
# 今日加工する用

In [61]:
from pandas import json_normalize
import pandas as pd
import json
import datetime
import glob
import re

In [ ]:
#今日の日付を4桁の数字で取得
today = pd.to_datetime(datetime.date.today()).strftime("%m%d")

# ファイルパスのパターンを定義
pattern = f"C:/Users/kyuza/.vscode/FILE/bleague_scraping/ryukyu/data/*_{today}_price.json"

# パターンに一致するファイルのリストを取得
files = glob.glob(pattern)

# gamedateの要素を抽出するためのリストを初期化
gamedates = []

# ファイル名からgamedateを抽出
for file in files:
    # ファイル名からgamedateを抽出する正規表現
    match = re.search(r"(\d+)_\d{4}_price\.json", file)
    if match:
        gamedates.append(match.group(1))

gamedates

In [ ]:
for gamedate in gamedates:
    #priceに関するjsonデータを取得
    filepath_price = f"C:/Users/kyuza/.vscode/FILE/bleague_scraping/ryukyu/data/{gamedate}_{today}_price.json"
    #sectionに関するjsonデータを取得
    filepath_section = f"C:/Users/kyuza/.vscode/FILE/bleague_scraping/ryukyu/data/{gamedate}_{today}_section.json"

    # JSONデータをPythonの辞書に変換
    data = pd.read_json(filepath_section)
    # 各セクションのpriceLevelとamountを抜き出す
    results = []
    for section, details in data.items():
        for price_info in details["availability"]["prices"]:
            result = {
                "section": section,
                "priceLevel": price_info["priceLevel"],
                "amount": price_info["amount"]
                }
            results.append(result)
            
        section_data = pd.DataFrame(results)


    # ファイルパス
        
    # ファイルを開き、JSONとして読み込む
    with open(filepath_price, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
    price_details = []
    for offer in data['offerPrices']:
        for zone in offer['zonePrices']:
            for level in zone['priceLevels']:
                for price in level['prices']:
                    price_details.append({
                        'game_info' : offer['offerLookupId'],
                        'game_info2' : offer["offerName"],
                        'priceLevelID': level['priceLevelID'],
                        'label': level['label'],
                        'priceTypeID': price['priceTypeID'],
                        'base': price['base']
                        })
                    
    # データを取得した日をdata_get_dateとする
    price_data = pd.DataFrame(price_details)
    price_data["data_get_date"] = datetime.date.today()
    price_data["data_get_date"] = pd.to_datetime(price_data["data_get_date"])
        
    # game_info内の連続する数列8つをgame_dayとする
    price_data['game_date'] = price_data['game_info'].str.extract(r'(\d{8})')
    price_data['game_date'] = pd.to_datetime(price_data['game_date'], format='%Y%m%d')

    # home_teamを追加
    price_data["home_team"] = "琉球"
    
    # game_info内のvsの後の文字をaway_teamとする
    price_data['away_team'] = price_data['game_info2'].str.extract(r'vs([^\s]+)')
    
    # priceTypeIDを購入者区分に置き換える
    price_data["priceTypeID"].replace({
        "650": "大人・こども共通",
        "651": "大人",
        "652": "こども"
        }, inplace=True)
    
    section_data

    # priceデータとsectionデータを一つに結合
    try:
        pl = pd.merge(section_data, price_data, left_on='priceLevel', right_on='priceLevelID', how='inner')
        # 100倍になっている値段を修正
        pl["base"] = pl["base"] / 100
        # カラム名を分かりやすい名前に修正
        pl = pl.rename(columns={
            "priceLevel" : "席種ID",
            "amount" : "残り席数",
            "label" : "席種名",
            "priceTypeID" : "販売者区分",
            "base" : "値段"
            })
        
        # 必要ないカラムを削除
        pl = pl.drop(columns=["席種ID","game_info","game_info2","priceLevelID"])
        
        data_get_date_filename, game_date_filename = pl["data_get_date"][0].strftime("%m%d"), pl["game_date"][0].strftime("%m%d")
        pl.to_csv(f"{game_date_filename}_{data_get_date_filename}_ryukyu.csv",encoding="shift-jis",index=False)
    except KeyError:
        pass

# 売り切れの日を把握できるようにする
* 例えば、売り切れになった日をpythonで処理できるようにしとく

***
# 以前の日付のものを加工する用

In [62]:
#取得したい日付を4桁の数字で取得
today = "0430"

# ファイルパスのパターンを定義
pattern = f"C:/Users/kyuza/.vscode/FILE/bleague_scraping/ryukyu/data/*_{today}_price.json"

# パターンに一致するファイルのリストを取得
files = glob.glob(pattern)

# gamedateの要素を抽出するためのリストを初期化
gamedates = []

# ファイル名からgamedateを抽出
for file in files:
    # ファイル名からgamedateを抽出する正規表現
    match = re.search(r"(\d+)_\d{4}_price\.json", file)
    if match:
        gamedates.append(match.group(1))

gamedates

['0510', '0511', '0513']

In [63]:
for gamedate in gamedates:
    #priceに関するjsonデータを取得
    filepath_price = f"C:/Users/kyuza/.vscode/FILE/bleague_scraping/ryukyu/data/{gamedate}_{today}_price.json"
    #sectionに関するjsonデータを取得
    filepath_section = f"C:/Users/kyuza/.vscode/FILE/bleague_scraping/ryukyu/data/{gamedate}_{today}_section.json"

    # JSONデータをPythonの辞書に変換
    data = pd.read_json(filepath_section)
    # 各セクションのpriceLevelとamountを抜き出す
    results = []
    for section, details in data.items():
        for price_info in details["availability"]["prices"]:
            result = {
                "section": section,
                "priceLevel": price_info["priceLevel"],
                "amount": price_info["amount"]
                }
            results.append(result)
            
        section_data = pd.DataFrame(results)


    # ファイルパス
        
    # ファイルを開き、JSONとして読み込む
    with open(filepath_price, 'r', encoding='utf-8') as file:
        data = json.load(file)
        
    price_details = []
    for offer in data['offerPrices']:
        for zone in offer['zonePrices']:
            for level in zone['priceLevels']:
                for price in level['prices']:
                    price_details.append({
                        'game_info' : offer['offerLookupId'],
                        'game_info2' : offer["offerName"],
                        'priceLevelID': level['priceLevelID'],
                        'label': level['label'],
                        'priceTypeID': price['priceTypeID'],
                        'base': price['base']
                        })
    price_data = pd.DataFrame(price_details)
    
    # データを取得した日をdata_get_dateとする
    year_string = datetime.datetime.now().year
    formatted_date_string = f"{year_string}{today}"
    price_data["data_get_date"] = pd.to_datetime(formatted_date_string,format='%Y%m%d')
        
    # game_info内の連続する数列8つをgame_dayとする
    price_data['game_date'] = price_data['game_info'].str.extract(r'(\d{8})')
    price_data['game_date'] = pd.to_datetime(price_data['game_date'], format='%Y%m%d')

    # home_teamを追加
    price_data["home_team"] = "琉球"
    
    # game_info内のvsの後の文字をaway_teamとする
    price_data['away_team'] = price_data['game_info2'].str.extract(r'vs([^\s]+)')
    
    # priceTypeIDを購入者区分に置き換える
    price_data["priceTypeID"].replace({
        "650": "大人・こども共通",
        "651": "大人",
        "652": "こども"
        }, inplace=True)
    
    price_data

    # priceデータとsectionデータを一つに結合
    try:
        pl = pd.merge(section_data, price_data, left_on='priceLevel', right_on='priceLevelID', how='inner')
        # 100倍になっている値段を修正
        pl["base"] = pl["base"] / 100
        # カラム名を分かりやすい名前に修正
        pl = pl.rename(columns={
            "priceLevel" : "席種ID",
            "amount" : "残り席数",
            "label" : "席種名",
            "priceTypeID" : "販売者区分",
            "base" : "値段"
            })
        
        # 必要ないカラムを削除
        pl = pl.drop(columns=["席種ID","game_info","game_info2","priceLevelID"])
        
        data_get_date_filename, game_date_filename = pl["data_get_date"][0].strftime("%m%d"), pl["game_date"][0].strftime("%m%d")
        pl.to_csv(f"{game_date_filename}_{data_get_date_filename}_ryukyu.csv",encoding="shift-jis",index=False)
    except KeyError:
        pass


C:\Users\kyuza\AppData\Local\Temp\ipykernel_18568\2347588872.py:8: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  data = pd.read_json(filepath_section)
C:\Users\kyuza\AppData\Local\Temp\ipykernel_18568\2347588872.py:8: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  data = pd.read_json(filepath_section)
C:\Users\kyuza\AppData\Local\Temp\ipykernel_18568\2347588872.py:8: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, str